In [1]:
# Imports
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.multivariate.manova import MANOVA
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from sklearn.decomposition import PCA
sys.path.append('src/')
from utils.methods import *
from utils.plots import *
from data.data_processing import *
from models.manova import *
from models.timeseries import *
from models.regression import *

In [2]:
# Set a consistent style for the plots
sns.set_theme(style="whitegrid")

In [3]:
# Prefix path
PREFIX_PATH = 'data'

In [4]:
# Set to display all columns
pd.set_option('display.max_columns', None)

# Set to display 50 rows
pd.set_option('display.max_rows', 50)

# Importing the final dataset
df = pd.read_csv(f'{PREFIX_PATH}/final_data_complete.csv')

In [7]:
df['Languages'] = df['Languages'].apply(str_to_list)
df['Genres'] = df['Genres'].apply(str_to_list)
# df["distilbert_emotions"] = df["distilbert_emotions"].apply(conv_to_dict)
df['merge_year'] = df['merge_year'].astype('Int64')
emotion = pd.DataFrame(list(df['distilbert_emotions'].apply(conv_to_dict)))
emotions_df = pd.concat([df, emotion], axis=1)

ValueError: malformed node or string: ['English Language']

In [8]:
df.head()

,WikiID,FreebaseID,Name,ReleaseDate,Runtime,Languages_tuple,Countries_tuples,Genres_tuples,Languages,Countries,Genres,Plot,merge_year,nrclex_emotions,roberta_emotions,distilbert_emotions,CharacterName,ActorAge,ActorGender,ActorBirthDate,ActorGenderFlag,merge_title,movieId,title,mean_ratings,std_dev_ratings,count_ratings
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",[English Language],['United States of America'],"[Thriller, Science Fiction, Horror, Adventure,...","Set in the second half of the 22nd century, th...",2001,"{'fear': 0.16296296296296298, 'anger': 0.13333...","{'neutral': 0.8483592867851257, 'realization':...","{'disgust': 0.5021221041679382, 'fear': 0.3510...","['Akooshay', 'Lieutenant Melanie Ballard', 'De...","[42.0, 27.0, 32.0, 33.0, 23.0, 52.0, 56.0, 37....","['F', 'F', 'M', 'M', 'F', 'F', 'F', 'M', 'M', ...","[Timestamp('1958-08-26 00:00:00'), Timestamp('...","[1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",ghosts of mars,4735.0,Ghosts of Mars (2001),2.427083,1.087064,144.0
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",[English Language],['United States of America'],"[Mystery, Biographical film, Drama, Crime Drama]",NaN,2000,NaN,NaN,NaN,"['Police Officer', 'Reporter', nan, nan, 'FBI ...","[37.78852253927453, 43.0, 37.78852253927453, 3...","['M', 'F', 'M', 'F', 'M', 'M', 'M', 'F', 'F', ...","[NaT, Timestamp('1956-12-19 00:00:00'), NaT, N...","[0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]",getting away with murder: the jonbenét ramsey ...,NaN,NaN,NaN,NaN,NaN
2,28463795,/m/0crgdbh,Brun bitter,1988,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",[Norwegian Language],['Norway'],"[Crime Fiction, Drama]",NaN,1988,NaN,NaN,NaN,"[nan, nan, nan, nan]","[33.0, 28.0, 40.0, 30.0]","['M', 'F', 'M', 'F']","[Timestamp('1954-10-05 00:00:00'), Timestamp('...","[0, 1, 0, 1]",brun bitter,NaN,NaN,NaN,NaN,NaN
3,9363483,/m/0285_cd,White Of The Eye,1987,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",[English Language],['United Kingdom'],"[Thriller, Erotic thriller, Psychological thri...",A series of murders of rich young women throug...,1987,"{'fear': 0.1751412429378531, 'anger': 0.135593...","{'neutral': 0.925209254026413, 'approval': 0.0...","{'anger': 0.5024036765098572, 'disgust': 0.324...","[nan, nan]","[32.0, 26.0]","['M', 'F']","[Timestamp('1954-05-08 00:00:00'), Timestamp('...","[0, 1]",white of the eye,90015.0,White of the Eye (1987),2.250000,2.474874,2.0
4,261236,/m/01mrr1,A Woman in Flames,1983,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",[German Language],['Germany'],[Drama],"Eva, an upper class housewife, becomes frustra...",1983,"{'fear': 0.0673076923076923, 'anger': 0.105769...","{'neutral': 0.8632992506027222, 'approval': 0....","{'anger': 0.3256497378461063, 'disgust': 0.163...","[nan, nan, nan]","[32.0, 32.0, 35.0]","['F', 'M', 'M']","[Timestamp('1950-06-20 00:00:00'), Timestamp('...","[1, 0, 0]",a woman in flames,NaN,NaN,NaN,NaN,NaN


In [16]:
titanic = df[(df["Name"] == "Titanic") & (df["merge_year"] == 1997)]
titanic_emotions = titanic["distilbert_emotions"]
titanic

,WikiID,FreebaseID,Name,ReleaseDate,Runtime,Languages_tuple,Countries_tuples,Genres_tuples,Languages,Countries,Genres,Plot,merge_year,nrclex_emotions,roberta_emotions,distilbert_emotions,CharacterName,ActorAge,ActorGender,ActorBirthDate,ActorGenderFlag,merge_title,movieId,title,mean_ratings,std_dev_ratings,count_ratings
22266,52371,/m/0dr_4,Titanic,1997-11-01,194.0,"{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","{""/m/09c7w0"": ""United States of America""}","{""/m/0fx2s"": ""Tragedy"", ""/m/04xvh5"": ""Costume ...","[Italian Language, English Language, French La...",['United States of America'],"[Tragedy, Costume drama, Historical fiction, A...","In 1996, treasure hunter Brock Lovett and his...",1997,"{'fear': 0.05726872246696035, 'anger': 0.04845...","{'neutral': 0.7688653270403544, 'realization':...","{'sadness': 0.2629161837200324, 'fear': 0.0365...","['Rose DeWitt Bukater', 'Jack Dawson', 'Caledo...","[22.0, 22.0, 31.0, 49.0, 45.0, 87.0, 42.0, 52....","['F', 'M', 'M', 'F', 'F', 'F', 'M', 'M', 'M', ...","[Timestamp('1975-10-05 00:00:00'), Timestamp('...","[1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",titanic,1721.0,Titanic (1997),3.402162,1.133026,3700.0


In [17]:
titanic_emotions

22266    {'sadness': 0.2629161837200324, 'fear': 0.0365...
Name: distilbert_emotions, dtype: object